In [2]:
import numpy as np
import yaml

with open("siso_mimo_config.yml","r") as f:
    data=yaml.safe_load(f)

EsN0dB=data['EsN0dB']
EbN0dB=data['EbN0dB']
N0=data['N0']
EsN0dB_fine=data['EsN0dB_fine']
EbN0dB_fine=data['EbN0dB_fine']
N0_fine=data['N0_fine']
Es=data['symbol_energy']


<function svd at 0x108bc6870>